In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## 1. User Model 
In the first part of this notebook, we built up the user model part for our model-based approach to the recommendation problem. The user model consists of two parts -- a state representation model and a reward prediction model. The goal is to train such a model that takes as input a user's historical record $(a_1,r_1, ..., a_{t-1},r_{t-1},a_t)$ and output/predict what the user's feedback r_t will be. Formally, try to train a model $\hat{r}$ parameterized by $\theta$ such that $\hat{r}_\theta(o_{t-1},a_t) \approx r(o_{t-1}, a_t)$.

### State Representation Model
We used a RNN layer to encode historically records of an user, $(a_1,r_1, a_2,r_2,..., a_{t-1},r_{t-1})$, into a fixed-length vector which we call $S_t$. Notice that the size of the input of the RNN (user's historical records) gradually increases over time so we decided to use a RNN to convert it into a fixed-length vector.

### Reward Prediction Model
After getting our state representation of the user's history, we concatenate it with the user's latest action a_t and feed it into a simple fully-connected layer to output our prediction of $\hat{r}_t$.

### Item/Action Embedding
One extra thing we did is to embed each item (or action) in the store into a vector in some high-dimensional space. For example, say we have in total 10,000 items in the repository, and we index each item with a number between 1 and 10,000. When we have a user's historical records and want to feed into the model, we need to feed all those actions $(a_1, a_2, ..., a_{t-1}, a_t)$ into a embedding layer first.



Since we are going to train the user model in a supervised manner with all users' historical records, what we are going to do first is to implement how to do this with *one* user's historical records -- $(a_1, r_1, ..., a_t, r_t)$. 

Let's get started!

In [1]:
%env PYTHONHASHSEED=0

import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, SimpleRNN, Concatenate, Lambda, Softmax
from keras.layers.embeddings import Embedding
import keras.backend as K
import numpy as np
import scipy.stats
from collections import deque
from scipy.sparse import coo_matrix
from copy import deepcopy
import time
import random as rn

# For strict reproducible purpose
np.random.seed(42)
rn.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

env: PYTHONHASHSEED=0


Using TensorFlow backend.


### Data Preprocessing
Preprocess the raw input data from some user $(a_1, r_1, ..., a_t, r_t)$ into a bunch of input-output pairs where input is $(a_1,r_1,...,a_i)$ and output is $r_i$ for all $i$ in $[1,t]$.

- We will first slice it into different input-output pairs:
    - $a_1 \rightarrow r_1$
    - $a_1, r_1, a_2 \rightarrow r_2$
    - ...
    - $a_1, r_1, ..., a_t \rightarrow r_t$

(The next two steps are intended for matching up the input/output shape of the Keras model we are going to build next. Feel free to skip it now :))
- Secondly, for each input-output pair, say, $a_1, r_1, ..., a_7, r_7, a_8 \rightarrow r_8$, I need to extract two inputs and one output from it which will later be fed into the model:
    - input1 = $0, a_1, a_2, ..., a_7, a_8$
    - input2 = $0, r_1, r_2, ..., r_7$
    - output = $r_8$

- Finally, convert all those lists (input1, input2, output) into numpy arrays and reshape them to be the correct shape for the model

In [2]:
# Convert the user's observation into strucutred format
# that can later be fed into the reward prediction model
# to either predict reward or get state representation
# (NOTE: this function works fine even if observation is an empty list. This happens
#  when we want to get the initial state represenation of the MDP.)
def obs2modelinputs(observation):
    '''
    params: observation: a list of user's historical records [a_1,r_1,...,a_{t-1},r_{t-1}] (for state representation purpose)
                         or [a_1,r_1,...,a_{t-1},r_{t-1},a_t] (for reward prediction purpose)
    
    return: observation's corresponding structured data that can be directly fed into the reward prediction model
    '''
    
    if len(observation) % 2 == 0:
        # To get state representation, we simply append a 0 to observation as a 'fake' a_t
        # This is okay since we won't use a_t anyway when computing the state representation of observation
        action_inputs = np.array([0] + observation[::2] + [0]) 
    else:
        action_inputs = np.array([0] + observation[::2])
    
    reward_inputs = np.array([0] + observation[1::2])
    
    # Reshape inputs so that the reward prediction model
    # or the state representation model can accpet
    action_inputs = action_inputs.reshape(1,-1)
    reward_inputs = reward_inputs.reshape(1,-1)
    
    return action_inputs, reward_inputs

In [3]:
# Construct training set -- a list of training samples obs_{t-1},a_t --> r_t, t = {1,2,...T}
# in strucutured format -- from one user's current historical records [a_1,r_1,...,a_T,r_T]
# The return can directly be sampled one at a time and feed into the reward prediction for training
def construct_strucutred_training_set(observation):
    '''
    param:
    observation: a list of this user's interactions with the recommendation system
                 [a_1,r_1,...,a_T,r_T]
    
    return:
    structured_training_set: a list of training samples where each element in this list is
    a tuple (inputs, outputs) for that training sample.
    '''
    
    assert len(observation)%2 == 0, 'Please make sure the observation on which \
                                     you want to construct training set has even number of elements.'
    
    T = len(observation) // 2
    # training_set stores all the training samples sliced from the observation
    # each training sample in it is a tuple ([a_1,r_1,...,a_t], r_t) for t = {1,2,...T}
    training_set = [] 
    for t in range(T):
        x = observation[0 : 2 * t + 1] # a list   [a_1,r_1,...a_t]
        y = observation[2 * t + 1]     # a scalar r_t
        training_set.append((x, y))
    
    # Convert each training sample in training_set into structured format
    structured_training_set = []
    for sample in training_set:
        x = sample[0] # [a_1,r_1,...,a_t]
        y = sample[1] # r_t
        
        action_inputs, reward_inputs = obs2modelinputs(x)
        # Do a similar thing to output/r_t
        reward_output = np.array(y).reshape(1,1)
        
        structured_training_set.append(([action_inputs, reward_inputs], [reward_output]))
    
    return structured_training_set


### Generator function 
it generats one batch of training samples at a time for the *reward prediction model* to train on.

In [4]:
def training_sample_generator(structured_training_set):
    '''
    params: 
    structured_data: Return of the construct_strucutred_training_set function. 
    '''
    while True:
        # Randomly yield one training sample from the training set without repetition
        # before consuming all training samples and starting a new round
        np.random.shuffle(structured_training_set)
        for training_sample in structured_training_set:
            yield training_sample

### Building-up layers for the user model
We will define the necessary layers required for our user model. 
Later, we will build two models, reward prediction model and policy model, from those layers.

In [5]:
num_users = 69878 # total number of users -- used in the environment simulator
num_items = 10677 # total number of items -- cardinality of action space |A|
dimension_action_embedding = 50
dimension_state_representation = 10
gamma = 0.99 # Discounting factor

r_low = 0
r_high = 5
dimension_reward_ohe = 10

# Three functions that will later be used to construct Lambda layers
def tminusone_actions(action_embeddings):
    # action_embeddings.shape = (batch_size, time_steps, dimension_action_embedding)
    return action_embeddings[:,:-1,:]

def action_t(action_embeddings):
    # action_embeddings.shape = (batch_size, time_steps, dimension_action_embedding) 
    return action_embeddings[:,-1,:]

def one_hot(reward_inputs):
    # reward_inputs.shape = (batch_size, time_steps)
    # We implicitly use broad-casting here since dimension_reward_ohe, r_low, r_high are all scalars
    indices = dimension_reward_ohe - (dimension_reward_ohe * (r_high - reward_inputs)) // (r_high - r_low)
    
    # indices is a rank-2 tensor whose datat type is floating point although each
    # element in indices is already integer (e.g. 3.0, 8.0, 9.0, etc.). We thus cast
    # it to integer data type so that one_hot function call be evoked correctly
    indices = tf.dtypes.cast(indices, 'int32')
    
    # shape of return = (batch_size, time_steps, dimension_reward_ohe)
    return K.one_hot(indices, dimension_reward_ohe)

In [6]:
# 2 Input Layers (notice the slightly asymmetry b/t these two inputs)
action_inputs = Input(shape=(None,), name='action_inputs', dtype='int32') # 0,a_1,a_2,...,a_{t-1},a_t. 'None' here means any t can be expected
reward_inputs = Input(shape=(None,), name='reward_inputs', dtype='float32') # 0,r_1,r_2,...,r_{t-1}.     'None' here means any t can be expexted


# Embedding layer for actions
# input_dim = num_items + 1 because items are ranked from 1 to num_items. We reserve 0 for the dummy action
action_embeddings = Embedding(input_dim=num_items+1, output_dim=dimension_action_embedding, name='action_embeddings')(action_inputs)

# Split the output to two parts -- embeddings for (0,a_1,...,a_{t-1}) and embedding for a_t
tminusone_action_embeddings = Lambda(tminusone_actions)(action_embeddings)
action_t_embedding = Lambda(action_t)(action_embeddings)

# One-hot encoding reward inputs
t_minusone_reward_ohes = Lambda(one_hot)(reward_inputs)

# RNN layer for state representation
rnn_inputs = Concatenate(axis=-1)([tminusone_action_embeddings, t_minusone_reward_ohes])
state_representation = SimpleRNN(dimension_state_representation, name='state_representation')(rnn_inputs)


# A dense layer for reward prediction
reward_prediction_inputs = Concatenate(axis=-1)([state_representation, action_t_embedding])
reward_prediction = Dense(1)(reward_prediction_inputs)

#### Reward Prediction Model
The reward prediction model consist of an action/item embedding layer + a RNN (aka state representation layer) + a reward prediction model (fully-connected layer).

The inputs of training data for this model are of various length and we have only one training sample for each length. Thus, we feed each training sample to the model and train the network on each sample (like stochastic gradient descent). 

In [7]:
# Reward prediction model
reward_prediction_model = Model(inputs=[action_inputs, reward_inputs], outputs=reward_prediction)


# 'Freeze' the state-representation model for training the
# state-value prediction model and the policy model.
# NOTICE: we freeze the state representation weights only because
# we want to use the same initial weights (trained and saved by
# the model-free solution) for both model-free and model-based solution
reward_prediction_model.get_layer('action_embeddings').trainable = False
reward_prediction_model.get_layer('state_representation').trainable = False


# Model compilation: 'rmsprop' is recommended for rnn and 'mse' is used 
# because we have a regression problem
reward_prediction_model.compile(optimizer='rmsprop', loss='mse')

# Train the reward prediction model on user's observation 
def train_reward_prediction_model(observation, steps_per_epoch=1000, epochs=10):
    '''
    params: observation: a list of user's current historical records [a_1,r_1,...,a_t,r_t]
            steps_per_epoch: steps to train in each epoch
            epochs: number of total training epochs
    '''￼
    structured_training_set = construct_strucutred_training_set(observation)
    reward_prediction_model.fit_generator(training_sample_generator(structured_training_set), steps_per_epoch=steps_per_epoch, epochs=epochs, verbose=0)

# Predict the user's feedback/reward for action based on his/her historical records (observation)
def predict_reward(observation, action):
    '''
    params: observation: a list of user's historical records [a_1,r_1,...,a_{t-1},r_{t-1}]
            action: the action just taken a_t (a scalar)
            
    return: predicted reward for the item just recommended \hat{r}_t (a scalar)
    '''
    action_inputs, reward_inputs = obs2modelinputs(observation + [action])
    return reward_prediction_model.predict([action_inputs, reward_inputs])[0][0]

#### State Representation Model
It turns out that we are going to need to compute the current state $S_t$ given the user's current historical record $O_{t-1}$. Thus, let's define a state representation model which we can conveniently use to encode history into states.

In [8]:
# Well, this state representation model is not necessary.
# We still define such a model for debugging convenience
# since we might want to take a look at what the state 
# representation is like.
state_representation_model = Model(inputs=reward_prediction_model.input, outputs=reward_prediction_model.get_layer(name='state_representation').output)

def get_state_representation(observation):
    '''
    params: observation: a list of user's historica records to be encoded [a_1,r_1,...,a_{t-1},r_{t-1}]
    
    return: an encoded vector representing S_t. Shape = (1, dimension_state_representation).
    '''
    assert len(observation) % 2 == 0, 'historical records to be encoded to a state must have even length.'
    
    action_inputs, reward_inputs = obs2modelinputs(observation)
    return state_representation_model.predict([action_inputs, reward_inputs])

##### Sanity Check 1
A sanity check to see whether the code is working by playing with some toy contrived data.

Well, at least I saw two things:
    1. The program can run without complilation errors;
    2. The reward prediction model overfits the toy dataset;

There is one things I don't know how to verify:
    1. How to verify we learned a good state representaion? Given an observation $O_{t-1}$, I can feed it to the state representation network and print out the result but I don't know how to intepret those numbers. How can I somehow find a way to justify that indeed those state representation is good.

In [ ]:
partial_observation = [123,5, 12,8, 291, 10, 244, 4, 2521,1]
print('State representation of observation before training the state representation model')
print(get_state_representation(partial_observation))
print('Now take action 5771')
print('Based on this state representation, our prediction is: ')
print(predict_reward(partial_observation, 5771))

observation = [123,5, 12,8, 291, 10, 244, 4, 2521,1, 5771,10, 6811,9, 2349, 2, 19, 1, 52,5, 122,5, 1272,8, 99,6]
train_reward_prediction_model(observation)


print('State representation of observation after training the state representation model')
print(get_state_representation(partial_observation))
print('Based on this state representation, our prediction is: ')
print(predict_reward(partial_observation, 5771))

## 2. One-step Actor-Critic to Improve our recommendation policy
Now we are going to build up the policy network. After receiving each transition $(S_t,A_t,R_t,S_{t+1})$, we can use Actor-Critic algorithm (one-step) to update our policy network as well as our state-value prediction network.

We chose the *One-step Actor-Critic* algorithm in RL to update the policy network. Namely, the algorithm requires the policy to chose an action $A_t$ in a given state $S_t$ (computed from $O_{t-1}$). Take this action and observe reward $R_t$ and next state $S_{t+1}$ (computed from $O_t$). Next we compute the temporal-difference error $\delta=R_t + \gamma*\hat{v}(S_{t+1};w) - \hat{v}(S_t;w)$, where $\hat{v}(S;w)$ is a network (function approximator) parameterized by $w$ that tries to predict state-values. With this TD error, we can update the state-value prediction network and later the policy network. The details of how to update those two networks are discussed later. Now let's first try to build up a state-value prediction network.

### State-value Prediction Network
We implement a simple fully connected layer to predict the state-value of a given state encoded from observation: $\hat{v}(O_{t-1};w) \approx v(S_t)$.

#### Training the State-value Prediction Network with One-Step Transition
After receiving the one-step transition $(S_t,A_t,R_t,S_{t+1})$, we train/update the state-value prediction network as follows:
- first compute the temporal-difference error $\delta = R_t + \gamma*\hat{v}(S_{t+1};w)$;
- secondly, fit the model with (one) training sample $(O_{t-1}, \delta)$ using 'mean_squared_error' loss. Furthermore, I choose 'sgd' optimizer since it is simple and we will be training this state-value prediction network with only one transition tuple anyway, meaning batch_size=1, so it does not make sense to use advanced optimizer which will only have an effect when there are more than more training sample in the batch.

In [9]:
# Build up fully connected layers (here it is just one layer) 
# after the state representation network to predict state-values
# of the state representation
state_value_prediction = Dense(1)(state_representation)

# # 'Freeze' the state-representation model for training the
# # state-value prediction model and the policy model
# reward_prediction_model.get_layer('action_embeddings').trainable = False
# reward_prediction_model.get_layer('state_representation').trainable = False


state_value_prediction_model = Model(inputs=[action_inputs, reward_inputs], outputs=state_value_prediction)
state_value_prediction_model.compile(optimizer='sgd', loss='mse')


def predict_state_value(observation):
    '''
    params: observation: a list of user's historica records to be encoded [a_1,r_1,...,a_{t-1},r_{t-1}]
    
    return: predicted state value of the encoded state from observation (a scalar)
    '''
    assert len(observation) % 2 == 0, 'historical records to be encoded to a state must have even length.'
    
    action_inputs, reward_inputs = obs2modelinputs(observation)
    return state_value_prediction_model.predict([action_inputs, reward_inputs])[0][0]
    
    
def train_state_value_prediction_model_with_one_step_transition(observation, action, reward):
    '''
    params: observation: the user's historical records [a_1,r_1,...,a_{t-1},r_{t-1}]
            action: the action a_t just taken by the recommendation system (a scalar)
            reward: user's feedback on the action/item just taken (a scalar)
    '''
    TD_error = reward + gamma * predict_state_value(observation + [action, reward])
    
    action_inputs, reward_inputs = obs2modelinputs(observation)
    # Reshape TD_error so that it can be fed to model.fit function
    TD_error = np.array(TD_error).reshape(1,-1)
    
    state_value_prediction_model.fit([action_inputs,reward_inputs], TD_error, verbose=0)

##### Sanity Check 2
This sanity check whether the state-value prediction model is working or not. Again, a same problem here is that we don't know how to verify the training results. We don't even know the ground truth $v_\pi(S)$. In the Actor-Critic, this state value prediction model is supposed to learn state-values of current policy (the Actor).
Nevertheless, we can see that our code is at least bug-free with this sanity check.

In [ ]:
observation = [123,5, 12,8, 291, 10, 244, 4, 2521,1, 5771,10, 6811,9, 2349, 2, 19, 1, 52,5, 122,5, 1272,8, 99,6]

ob = []
print('ob = [], with initial weights:')
print('state representation of ob = ' + str(get_state_representation(ob)))
print('predicted state value for this state representation = ' + str(predict_state_value(ob)))

print('\nnow we take action 123 and receive reward 5, and we train the state prediction model with this one-step transition')
train_state_value_prediction_model_with_one_step_transition(ob, 123, 5)
print('training finished!')

ob = [123, 5]
print('\nob = [123, 5]')
print('state representation of ob = ' + str(get_state_representation(ob)))
print('predicted state value for this state representation = ' + str(predict_state_value(ob)))

### Policy Network
Now we are ready to build up the policy $\pi_\theta(A|S)$ -- a network, parameterized by $\theta$ that takes the user's current state $S$ as input and output probabilities of choosing all possible actions $\pi(A|S)$.

We are gonna re-use the state representation network to encode observations into states for us, and we connect the output of the state repersentation network to two layers: one dense layer that computes the *preferences* for each action in this state followed by a *soft-max* layer that normalizes these preferences into probabilities.

The preference $h(S,A)$ for each possible action is computed through a linear combination of the state representation $h(S,A) = w_A^Tx(S)$

In [10]:
preferences = Dense(num_items, name='preferences')(state_representation)
probabilities = Softmax(name='probabilities')(preferences)

policy_model = Model(inputs=[action_inputs, reward_inputs], outputs=probabilities)
policy_model.compile(loss='categorical_crossentropy', optimizer='sgd')

#### Sample an Action from the Policy Network
Given a state representation $S_t$, which is encoded from observation $O_{t-1}$, we can do a forward pass to compute the probabilities of each action under current policy network (output of policy_model). Then we sample an action [1, num_items] from this probabilistic distribution.

In [11]:
# Compute the probabilities of taking each action under 
# current (stochastic) policy
def predict_policy_probabilities(observation):
    '''
    params: observation: a list of user's current historical records [a_1,r_1,...,a_{t-1},r_{t-1}]
    
    return: probabilities: a numpy array of size num_items where each element corresponds to the 
                           probability of choosing that action under current policy
    '''
    assert len(observation) % 2 == 0, 'historical records to be encoded to a state must have even length.'
    
    action_inputs, reward_inputs = obs2modelinputs(observation)
    probabilities = policy_model.predict([action_inputs, reward_inputs])[0]
    
    return probabilities

# Take an action from the (stochastic) policy
def sample_action_from_policy(observation):
    '''
    params: observation: a list of user's current historical records [a_1,r_1,...,a_{t-1},r_{t-1}]
    
    return: an action A_t to take according to the current policy network. (a scalar)
    '''
    probabilities = predict_policy_probabilities(observation)
    sampled_action = np.random.choice(np.arange(1, num_items+1), p=probabilities)
    
    return sampled_action

#### Training the Policy Network with One-step Transition
According to the One-step Actor-Critic (Episodic) algorithm, the policy update formula is $$\theta \leftarrow \theta + \alpha \gamma^t (R_t + \gamma\hat{v}_w(S_{t+1}) - \hat{v}_w(S_t)) \nabla_\theta(\ln\pi_\theta(A_t|S_t))$$

We made two somewhat unreasonable modifications for practical reasons:
1. Strictly speaking, this is an algorithm for episodic case and our recommendation problem is a continuing problem per se, and this update is for episodic problems. But nevertheless, we will still use it because the Action-Critic algorithm for continuing problem on the textbook uses average reward, something I don't think I have a good understanding. So we will still use this algorithm despite it might not be a perfect match.
2. We will also get rid of the $\gamma^t$ term. This changes the estimate of the gradient of the objective function ($J(\theta)$) an biased one but we will live with it, because otherwise, as t goes up, we are afraid that the update will be scaled too small such that no substantial change will be made to the parameters $\theta$.

Thus, the update for our policy model becomes 
$$ \delta = R_t + \gamma\hat{v}_w(S_{t+1}) - \hat{v}_w(S_t) $$
$$\theta \leftarrow \theta + \alpha \delta \nabla_\theta(\ln\pi_\theta(A_t|S_t))$$

Then, our job is to design a loss as well as an appropriate input-output from $(S_t,A_t,R_t,S_{t+1})$ such that when keras computes the gradients of the network's parameters $\theta$, it should equal to $\delta \nabla_\theta(\ln\pi_\theta(A_t|S_t))$.
We use $(x,y) = (S_t, \delta \text{one_hot}(A_t))$ as our training sample (we train the network on only one sample at a time which is constructed from the latest transition) and use 'categorical_crossentropy' loss: 
$$ -\sum_{i=1}^{|A|}y_i \ln \hat{y}_i = - \sum_{i=1}^{|A|} \delta \text{one_hot}(A_t)_i \ln \pi_\theta(A_i|S_t) = - \delta \ln \pi_\theta(A_t|S_t)$$

Finally, when we call model.fit() with one training sample $(x,y)$ fed in, it will automatically compute the gradient of this loss w.r.t. $\theta$ 
$$ \nabla_\theta ( - \delta \ln \pi_\theta(A_t|S_t)) = - \delta \nabla_\theta (\ln \pi_\theta(A_t|S_t)) $$
and update $\theta$ in the opposite direction of the gradient (in order to minimize the loss)
$$ \theta \leftarrow \theta - \alpha (- \delta \nabla_\theta (\ln \pi_\theta(A_t|S_t))) \\ \theta \leftarrow \theta + \alpha \delta \nabla_\theta (\ln \pi_\theta(A_t|S_t)) $$
which is exactly what we want! Yay!

In [12]:
def train_policy_model_with_one_step_transition(observation, action, reward):
    '''
    params: observation: the user's historical records [a_1,r_1,...,a_{t-1},r_{t-1}]
            action: the action a_t just taken by the recommendation system (a scalar)
            reward: user's feedback on the action/item just taken (a scalar)
    '''
    assert len(observation) % 2 == 0, 'historical records to be encoded to a state must have even length.'
    
    predicted_value_state = predict_state_value(observation)
    predicted_value_next_state = predict_state_value(observation + [action, reward])
    
    TD_error = reward + gamma * predicted_value_next_state - predicted_value_state
    
    # create an one-hot encoding of action.
    # Do not confused it with the hot_encoding lambda layer.
    # Here it is just a trivial hot encoding of action.
    one_hot_encoding_action = np.zeros(num_items).reshape(1,-1) # Shape = (1,num_items)
    one_hot_encoding_action[0][action-1] = 1 # [action-1] because actions are indexed from 1 to num_items
    
    # a strange target, isn't it? See the documentation for more details!
    target = TD_error * one_hot_encoding_action
    
    # retrieve inputs
    action_inputs, reward_inputs = obs2modelinputs(observation)
    
    policy_model.fit([action_inputs, reward_inputs], target, verbose=0)

#### Sanity Check 3
Play with the policy network with contrievd data. Namely, we create a 'fake' environment simulator that returns reward given state and action. We then update the policy network with fixed state-value prediction network (with its initial weights) after receiving each transition tuple.

What I saw from the result is that, despite a very very bad state-value prediction network, the policy network converges to a deterministic policy and outputs the same action no matter what current state is. This is reasonable because the way how we set up the simulated environment does not use any state information. One thing I don't know whether it's reasonable or not is that the algorithm does not find the exact optimal action, which should be num_items//2 in our contrived example, but rather an item near it. Furthermore, the action this algorithm converges to is not the same between different trials.

In [ ]:
# Fake environment simulator
def simulated_reward(ob, action):
    # a Gaussian-like distribution of reward over items
    # Items with intermediate index (~num_items//2) have largest reward
    return scipy.stats.norm(num_items // 2, 1000).pdf(action) * 10000
    
ob = deque(maxlen=10)

for t in range(30000):
    print('\nt = ' + str(t))
    action = sample_action_from_policy(list(ob))
    print('a' + str(t+1) + ' = ' + str(action))
    reward = simulated_reward(list(ob), action)
    print('r' + str(t+1) + ' = ' + str(reward))
    print('Update policy')
    train_policy_model_with_one_step_transition(list(ob), action, reward)
    ob += [action, reward]

### One-Step Actor-Critic Algorithm
We have built all the components needed to implement the one-step Actor-Critic algorithm. We will now integrate them into one single function `one_step_actor_critic` that takes as input one transition tuple $(S_t, A_t, R_t, S_{t+1})$ (well, actually $(O_{t-1}, A_t, R_t)$ since $S_t$ can be encoded from $O_{t-1}$ and $S_{t+1}$ can be encoded from $O_t = [O_{t-1},A_t,R_t]$). This function does two things: update the state-value prediction network (Critic) and update the policy network (Actor).

In [13]:
def one_step_actor_critic(observation, action, reward):
    '''
    params: observation: the user's historical records [a_1,r_1,...,a_{t-1},r_{t-1}]
            action: the action a_t just taken by the recommendation system (a scalar)
            reward: user's feedback r_t on the action a_t (a scalar)
    
    '''
    train_state_value_prediction_model_with_one_step_transition(observation, action, reward)
    train_policy_model_with_one_step_transition(observation, action, reward)

## 3. Environment Simulator
Now we have every component done and are ready to test our model/algorithm with experiment. Since our algorithm concerns the problem of learning a good recommendation policy via interaction with the user, we will try to test how good the performance of our model/algorithm is by measuring the learning process with interactive recommendation experience with user. However, one big problem is that it is commercially impractical to test our algorithm by putting our recommendation model/algorithm online and let it interact with real customers. Therefore, we build an environment simulator that tries to mimic the customer's feedback/reward on recommended items based on public recommendation dataset. This environment simulator takes a user_id (an int) and a item_id (an int) as input and returns the user's feedback (a float) on this item.

### Dataset -- MovieLens 10M
Here we briefly discuss the dataset we used to build the environment simulator. We used the *MovieLens 10M Dataset* which contains 10,000,054 ratings for 10677 movies from 69878 users. The ratings range from 0.0 (exclusive) to 5.0 (inclusive) with an increment of 0.5. It also contains other information like tags of movies and time stamps but we will only use the ratings information (i.e. ratings for different movies from different users).

For example, among all those 10,000,054 ratings, one rating is like 46::152::3.5, which stands for user_46 gave a 3.5 to movie_152. Both the index of users and movies start from 1 to 69878 and 10677 respectively. 

On average, each user has 143 ratings (to 143 different movies), and each movie is rated by 936 users.

### Building Up a Simulator from MovieLens 10M
The idea is simple: we are going to create a large rating table whose rows represents user ids (from 1 to 69878) and columns represents movie ids (from 1 to 10677). If the dataset contains a rating $r$ for user_i to movie_j, then the $(i,j)$ entry of the table is $r$. We are also going to normalize the ratings from $(0.0, 5.0]$ to $(-1,1)$ while maintaining ratio:
$$new\_rating = \frac{2}{5} * old\_rating  - 1$$
And for those entries without actual rating in the dataset, we thus set it to 0.0, implying a neutral rating in the normalized range. So, you can see that this table is in fact a large sparse table with a lot of 0.0's. Well, there is nothing we can do about this since the dataset contains only this much ratings to fill a small part of the rating table.

One little extra thing is that in the original dataset, the ids of the users and movies are not indexed with {1,2,3,...} but might have gaps. That is, the actual index of users can be something like {1,2,3,5,6,9,10,...} so you might see a rating record from user_id 71567 although there are in total just 69878 users. Hence, I also reordered the user and movie index from {1,2,3,...} up 7to either 69878 for user_ids or 10677 for movie_ids through a simple mapping dictionary.

In [14]:

data_path = '/home/xiang/Desktop/Graduation_Thesis/ratings.dat'
data = np.loadtxt(data_path, delimiter='::')

rating_range_low = 0.0
rating_range_high = 5.0
new_rating_range_low = -1.0
new_rating_range_high = 1.0
# new_data = a * old_data + b transforms data from one range
# to another, which must be symmetric around 0, while maintaining ratio
assert new_rating_range_low + new_rating_range_high == 0, 'Normalized range must be symmetric around 0'
a = (new_rating_range_high - new_rating_range_low) / (rating_range_high - rating_range_low)
b = new_rating_range_low - a * rating_range_low      

user_ids = set()
movie_ids = set()

for user_id, movie_id, rating, time_step in data:
    user_ids.add(int(user_id))
    movie_ids.add(int(movie_id))

# Create the user ids mapping
# each key-value pair is (new_index : old_index_in_data)
user_ids_mapping = dict()
i = 1
for user_id in user_ids:
    user_ids_mapping[i] = user_id
    i += 1 

# Similarly, create the movie ids mapping
movie_ids_mapping = dict()
i = 1
for movie_id in movie_ids:
    movie_ids_mapping[i] = movie_id
    i += 1


# Create the rating table based on the (original) ratings
# in the dataset
rating_table = coo_matrix((data[:, 2], (data[:, 0].astype(int), data[:, 1].astype(int)))).todok()


def get_rating(user_id, movie_id):
    '''
    params: user_id: index of the user (an int from 1 to num_users inclusively)
            movie_id: index of the movie (an int from 1 to num_items inclusively)
    
    return: normalized simulated rating for this user_id to movie_id (a float b/t (-1, 1])
    '''
    old_rating = rating_table[user_ids_mapping[user_id], movie_ids_mapping[movie_id]]
    
    if old_rating == 0.0:
        return old_rating
    else:
        # normalize the rating to (-1, 1]
        return a * old_rating + b

### Putting All Together into the Dyna Architecture
Congrats on making to this point. We finished all the crucial components and one last thing is to put them together into the Dyna architecture.

This idea of Dyna architecture is model-based RL -- it constructs/learns a model of the environment dynamics and tries to learn from both real experience as well as imaginary experience generated from the model.

In [15]:
# Run the Dyna algorithm with user_id for max_iterations (T) rounds
# where within each round do the planning for max_planning_steps steps
def Dyna(user_id=1, max_iterations=100, max_planning_steps=100):
    '''
    params: user_id: the id of the user we are going to interact with in the (simulated)
                     environment. (an int between 1 and num_users inclusively)
            max_iterations: max number of iterations of the dyna algorithm (an int scalar)
            max_planning_steps: max number of steps allowed in the planning step
                                in the dyna algorithm (an int scalar)
                                
    return: observation: the interactive history (A_1,R_1,...,A_T,R_T), which will
                         be stored in a replay buffer and later used to periodically
                         retrain our user model.
    '''
    observation = []
    
    for t in range(max_iterations):
        # Choose an action to take from our policy model
        # and observe its reward
        action = sample_action_from_policy(observation)
        reward = get_rating(user_id, action)

        # Perform One-step Actor-Critic update with the latest transition
        one_step_actor_critic(observation, action, reward)

        observation += [action, reward]
        
        # Enter the planning module
        rollout_observation = deepcopy(observation)
        for _ in range(max_planning_steps):
            # Choose an action to take from our policy model
            # and query our reward prediction model for estimated reward
            action = sample_action_from_policy(rollout_observation)
            reward = predict_reward(rollout_observation, action)

            # Perform One-step Actor-Critic update with the latest imaginary transition
            one_step_actor_critic(rollout_observation, action, reward)

            rollout_observation += [action, reward]

    return observation

## Experiment
Finally, we are going to conduct some experiment with all the components we had built so far. From a high level, the experiment involves two parts:
1. Interact with the enviroment and learn a good recommendation policy;
2. Evaluate the learning process of our algorithm/model

We will go through each of the two parts in further detailed subparts.

### 1. Learn a Recommendation Policy through Interaction
We can break down this task further into three steps:

1. Pre-train a user model. We initialize a (random) policy $\pi_\theta(S,A)$ and use it to interact with the (simulated) environment and collect a set of interactive data $\{(A_1^{(1)},R_1^{(1)},\dots,A_T^{(1)},R_T^{(1)}), (A_1^{(2)},R_1^{(2)},\dots,A_T^{(2)},R_T^{(2)}), \dots, (A_1^{(K)},R_1^{(K)},\dots,A_T^{(K)},R_T^{(K)}) \}$, where $A_t^{(k)}$ represents the action taken in time step $t$ for user $k$ and $R_t^{(k)}$ accordingly stands for the reward/feedback from user $k$ for the recommended item $A_t^{(k)}$. We then use this dataset to train a user model. (K here is a hyperparameter)

2. Run the Dyna algorithm to learn a good recommendation policy. Start by randomly picking up an user and run the dyna algorithm with this user (`user_id`) for several steps (`max_iterations`, `max_planning_steps`). Add the return value (a list of (real) interactive history of this user) to a replay buffer, which is a fixed length deque of lists. Then we repeat the whole process by randomly selecting another user. We repeat this process until `buffer_ratio_retrain` ((0, 1] e.g. 1/3) data in the replay buffer is replaced by the new ones. Then we go to step 3.

3. Periodically retrain user model. Recall that we maintain a replay buffer (of fixed size) of interactive experiences. We will use the data in this buffer to retrain our user model.

In [16]:
def train_user_model_from_replay_buffer(replay_buffer):
    '''
    params: replay_buffer: a queue of interactive experience of same length.
    '''
    
    # Check for precondition -- make sure all elements in replay_buffer have the same length
    if len(replay_buffer) != 0:
        for idx in range(1, len(replay_buffer)):
            if len(replay_buffer[idx]) != len(replay_buffer[0]):
                assert False, 'historical experience in replay_buffer must have same length!'
    
    length_episode = len(replay_buffer[0]) // 2
    
    # Randomly select observations from replay_buffer without repetition
    # and train our user model on each observation. Since we don't want to
    # modify replay_buffer in-place, we create an auxiliary list for replay_buffer
    aux_replay_buffer = list(replay_buffer)
    np.random.shuffle(aux_replay_buffer)
    
    for ob in aux_replay_buffer:       
        train_reward_prediction_model(ob, steps_per_epoch=length_episode, epochs=1)
    

# Pre-train our user model by interacting with all users using
# an initial policy. This function does two things: fill in a 
# replay buffer and train our user model (aka reward prediction model)
# with data in the replay buffer
def pre_train_user_model(num_users_to_interact, length_episode):
    '''
    params: num_users_to_interact: number of users we are going to interact with (an int >= 1)
            length_episode: length of the interaction with each user (an int >= 1)
            
    return: a queue of length num_users_to_interact where each element is the interactive 
            history of that user.
    '''
    replay_buffer = deque(maxlen=num_users_to_interact)
    
    # Fill in the replay buffer
    for idx in range(num_users_to_interact):
        # Randomly select an user. Note that we index users from 1 to num_users!
        user_id = np.random.randint(num_users) + 1
    
        # Use the initial policy to interact with user user_id for length_episode
        # steps without improving the policy or anything -- just collect experiences
        ob = []
        for _ in range(length_episode):
            action = sample_action_from_policy(ob)
            reward = get_rating(user_id, action)
            ob += [action, reward]
        
        # Add this user's experience (ob) to replay buffer
        replay_buffer.append(ob)
    
    print('Fill in replay buffer finished!\n')
    
    # Train user model from the replay buffer
    train_user_model_from_replay_buffer(replay_buffer)
    
    print('Train on replay buffer finished!\n')
    
    return replay_buffer

### 2. Evaluate the Learning Process
Every once in a while, we need to evaluate our (recommendation) policy to see how good it current is. We are going to implement a procedure which can measure the performance of our policy using four metrics -- average reward, precision@k, recall@k, and F1@k.

Concretely, for **average reward**, here is how we are going to do it:
1. Randomly select an user $i$ from 1 to num_users;
2. Interact with this user (in the simulator) for certain steps (episode length e.g. 32) using current policy and record the rewards $R_1, R_2,\dots,R_{T}$;
3. Average of these $T$ rewards to calculate the average reward $\bar{R}^{(i)}$ for this user under current policy;
4. Go back to step 1 and repeat for multiple users, and average over all these average rewards $\bar{R}^{(i)}$'s to get the overall average reward $\bar{R}$.

For **precision@k** (we set k=32, same as episode length), here is how we are going to do it:
1. Randomly select an user $i$ from 1 to num_users;
2. Interact with this user (in the simulator) for certain steps (episode length e.g. 32) using current policy. 
   - During each step, look at the top-k (top-32) actions/items with highest probabilities. Then query the environment simulator to see how many of them have a reward of >= 0.2 (0.2 is the chosen threshould that corresponds 3 stars in our example). Then the proportion (e.g. 1/32 if only one of the 32 items with highest probabilities has an actual reward of >= 0.2) is the precision@k for this step. 
   - Repeat for 32 steps to compute 32 precision@k's and average over them to get the precision@k for this user.
3. Go back to step 1 and repeat for multiple users, and average over all these precision@k to get the overall precision@k.

For **recall@k** (k=32), this is how we are going to do it:
1. Randomly select an user $i$ from 1 to num_users;
2. Query the environment simulator for the reward about ALL the actions (from 1 to num_items) on this user $i$. Record those actions with a reward >= 0.2 (the same threshould as in precision@k); (minor note: If there is no such action (no relevant items), which I don't think will happen in this dataset, then the recall@k is set to 1 and done. No need to go to step 3.)
3. Interact with this user (in the simulator) for certain steps (e.g. 32) using current policy.
    - During each step, look at the top-k (top-32) actions/items with highest probabilities. Then query the environment simulator to see how many of them have a reward of >= 0.2 (0.2 is the chosen threshould that corresponds 3 stars in our example). Then the proportion (e.g. 1/134 if only one of the top-32 items with highest probabilities has an actual reward of >= 0.2 when 134 means this user $i$ has 134 relevant ratings) is the recall@k for this step.
    - Repeat for 32 steps to compute 32 recall@k's and average over them to get the recall@k for this user.
4. Go back to step 1 and repeat for multiple users, and average over all these recall@k to get the overall recall@k.

In [17]:
def evaluate_current_policy(threshould, num_users_to_interact, length_episode=32, k=32):
    '''
    params: threshould: threshould of reward/rating to distinguish relevant and recommended actions.
                        Must be in normalized scale!!!
            num_users_to_interact: number of users we will interact to average out the noiseness (an int >= 1)
            length_episode: number of steps allowed for interaction for each user (an int >= 1)
            k: k as used in precision@k, recall@k, and f1@k (an int >= 1)
    
    return: overall_average_reward (a float)
            overall_precision_at_k (a float)
            overall_recall_at_k (a folat)
            overall_f1_at_k (a float)
    '''
    # check pre-conditions
    assert 0 < threshould <= new_rating_range_high, 'Input threshould must be in normalized scale!!!'
    assert num_users_to_interact >= 1
    assert length_episode >= 1
    assert k >= 1
    
    
    
    overall_average_reward = 0.
    overall_precision_at_k = 0.
    overall_recall_at_k = 0.
    overall_f1_at_k = 0.
    
    for idx in range(num_users_to_interact):
        # Randomly select an user. Note that we index users from 1 to num_users!
        user_id = np.random.randint(num_users) + 1
        
        # Average metrics over multiple interaction steps
        average_reward = 0.
        precision_at_k = 0.
        recall_at_k = 0.
        f1_at_k = 0.
        
        num_relevant_actions = 0
        for action_id in range(1, num_items+1):
            if get_rating(user_id, action_id) >= threshould:
                num_relevant_actions += 1
        if num_relevant_actions == 0:
            recall_at_k = 1.
            print('Oops, be careful because user No.' + str(user_id) + ' does not have any relevant ratings!')
        
        
        ob = []
        for t in range(length_episode):
            # Compute average reward in an incremental way over multiple steps
            action = sample_action_from_policy(ob)
            reward = get_rating(user_id, action)
            average_reward += 1/(t+1) * (reward - average_reward)
            
            
            # Compute recommended actions (aka the k actions with highest probabilities)
            probabilities = predict_policy_probabilities(ob)
            # Pick out k actions/items with highest probabilities
            actions_with_ascending_probability = np.argsort(probabilities)
            recommended_actions = actions_with_ascending_probability[-k:] + 1 # remember that our actions are indexed from 1 to num_items!
            
            # number of recommended AND relevant actions, meaning how many actions (out the k actions)
            # also have a true rating >= threshould
            num_recommended_relevant_actions = 0 
            for action_id in recommended_actions:
                if get_rating(user_id, action_id) >= threshould:
                    num_recommended_relevant_actions += 1
                    
            # Compute average precision@k in an incremental way over multiple steps
            precision_at_k_this_step = num_recommended_relevant_actions / k
            precision_at_k += 1/(t+1) * (precision_at_k_this_step - precision_at_k)
            
            # Compute average recall@k in an incremental way over multiple steps
            recall_at_k_this_step = 1. # in case num_relevant_actions is 0
            if num_relevant_actions != 0:
                recall_at_k_this_step = num_recommended_relevant_actions / num_relevant_actions
                recall_at_k += 1/(t+1) * (recall_at_k_this_step - recall_at_k)
            
            # Compute average f1@k in an incremental way over multiple steps
            if (precision_at_k_this_step + recall_at_k_this_step) != 0.:
                f1_at_k_this_step = 2 * precision_at_k_this_step * recall_at_k_this_step / (precision_at_k_this_step + recall_at_k_this_step)
            else:
                f1_at_k_this_step = 0.
            f1_at_k += 1/(t+1) * (f1_at_k_this_step - f1_at_k)
            
            ob += [action, reward]
        
        
        overall_average_reward += 1/(idx+1) * (average_reward - overall_average_reward)
        overall_precision_at_k += 1/(idx+1) * (precision_at_k - overall_precision_at_k)
        overall_recall_at_k += 1/(idx+1) * (recall_at_k - overall_recall_at_k)
        overall_f1_at_k += 1/(idx+1) * (f1_at_k - overall_f1_at_k)
        
    return overall_average_reward, overall_precision_at_k, overall_recall_at_k, overall_f1_at_k

### 3. Final Experiment with Evaluation

Todo: Finish this cell (simply the documentation), add

In [18]:
def main_train(max_rounds, replay_buffer, buffer_ratio_retrain, length_episode, max_planning_steps=100):
    '''
    params: max_rounds: number of rounds for the training process -- one round stands for interacting and 
                        learning from multiple users and retrain user model once.
            replay_buffer: a fixed-length queue with elements being interactive experience from 
                           various user with the environment. Or, simply use the return of the
                           pre_train_user_model function. (a fixed-length deque);
            buffer_ratio_retrain: periodically retrain our user model (aka reward prediction model) when this proportion
                                  of replay buffer has been replaced with new interactive data (a float in (0,1])
            length_episode: number of (real) steps allowed for each user for interaction in the dyna algorithm (an int >=1);
            max_planning_steps: number of steps for planning for each user in the dyna algorithm (an int >=1).
    '''
    # check pre-conditions
    # 1. make sure it is a valid replay buffer
    if len(replay_buffer) == 0:
        assert False, 'Please fill in replay buffer first!'
    else:
        for idx in range(1, len(replay_buffer)):
            if len(replay_buffer[idx]) != len(replay_buffer[0]):
                assert False, 'historical experience in replay_buffer must have same length!'
    
    # 2. make sure length_episode is equal to the length of existing element in replay_buffer
    assert length_episode == (len(replay_buffer[0]) / 2), 'length_episode must have the same length as experiences in replay_buffer'
    
    
    
    periodically_retrain_steps = int(buffer_ratio_retrain * len(replay_buffer))

    for idx in range(max_rounds):
        print('\n\n' + str(idx) + 'th round of the main training process:')
        
        # Interact with multiple users (determined by periodically_retrain_steps)
        # and learn from them in a model-based manner
        for t in range(periodically_retrain_steps):
            user_id = np.random.randint(num_users) + 1

            ob = Dyna(user_id=user_id, max_iterations=length_episode, max_planning_steps=max_planning_steps)

            replay_buffer.append(ob)
            
            if (t+1)%100 == 0 or t == 0:
                print('    interaction with ' + str(t) + 'th user in this round finished')
        
        print('Retrain user model for the ' + str(idx) + 'th round.')
        
        # Periodically retrain user model from (updated) replay buffer
        train_user_model_from_replay_buffer(replay_buffer)
        print('Retraining user model is finished!')
        
        # Evaluate current policy
        average_reward, precision_at_k, recall_at_k, f1_at_k = evaluate_current_policy(threshould=0.19, num_users_to_interact=len(replay_buffer), length_episode=length_episode, k=length_episode)
        print('Performance of current policy:\nAverage Reward = ' + str(average_reward) + ' precision@k = ' + str(precision_at_k) + ' recall@k = ' + str(recall_at_k) + ' f1@k = ' + str(f1_at_k) + '\n')

#### Sanity Check 4a (model-free solution) (NOT RECOMMENDED; READ THIS CELL CAREFULLY IF YOU INSIST!)
Run experiments to get results!

I am running the model-free algorithm again (with pre-train step)!

I will save the pre-trained user model so that later, when I run model-based algorithm (remember to freeze the RNN weights for periodically retraining to stick to the same state representation. Now  I did not do that and it does not matter because we will not retrain the user model anyway), I will load this weights so 1. I don't need to pretrain again and 2. I have the same initial state representation.

Currently, the code is for the model-based solution. Just run each cell one by one (except for those sanity check ones and this one) to 4b to perform the model-based algorithm. 
If you wish to run the model-free algorithm (not recommended), do the folloing three modifications to the code:
1. comment out the freeze-weights codes in section "reward prediction model" and uncomment a same code block in section "state-value predictio network";
2. comment out the planning module in Dyna function;
3. comment out the periodically retrain statement (line 47) in the main_train function.

In [ ]:
replay_buffer_size = 3000
length_episode = 32

start_time = time.time()

# Pre-train the user model (state representation model + reward prediction model)
replay_buffer = pre_train_user_model(num_users_to_interact=replay_buffer_size, length_episode=length_episode)

print('Time to pre-train the user model is ' + str((time.time() - start_time) / 60) + ' minutes')
print('Save the weights of the user model to weights_pretrained_user_model.h5')
reward_prediction_model.save_weights('weights_pretrained_user_model.h5')


main_train(max_rounds=60, replay_buffer=replay_buffer, buffer_ratio_retrain=1.0, length_episode=32, max_planning_steps=5)

print('\n\nTotal time to run the program is ' + str((time.time() - start_time) / 60) + ' minutes')

#### Sanity Check 4b (model-based solution)
Having run the previous cell of code for the model-free solution, we are now ready to try out our model-based solution!

We need to modify the code in the following way:
1. comment out the freeze-weights codes in section "state-value predictio network" and move it to reward prediction model. We do this because, in the model-free solution, we have pre-trained the user model and saved the weights of the user model after pre-training step. In our model-based solution, for fair comparison, we want to start our user model with the same weight AND during later periodical re-training of the user model, we do not want to change the weights of the state representation model but only the reward prediction model. Thus, we will omit the pre-training step in model-based solution and only load the saved weights for our user model. In later periodical retraining of the user model, we will only update the reward prediction model since we have  frozen the weights of the state representation model at the very begining.

2. uncomment the planning module in dyna.

3. uncomment the periodically retain step in main function.

In [21]:
replay_buffer_size = 3000
length_episode = 32

start_time = time.time()

# Load (saved weights) for the user model to ensure same 
# initialization of the state representation.
reward_prediction_model.load_weights('weights_pretrained_user_model.h5')

# We don't have pre-train here to return a full replay buffer for us,
# which is required in the main function. We will just create one and stuff 
# it with replay_buffer_size meaningless lists. Those replay_buffer_size lists
# will be replaced by new interactive data in the first round before re-training
# the user model on the replay buffer, so this is okay!
replay_buffer = deque(maxlen=replay_buffer_size)
for _ in range(replay_buffer_size):
    fake_ob = []
    for _ in range(length_episode):
        fake_ob += [0,0]
    replay_buffer.append(fake_ob)

main_train(max_rounds=60, replay_buffer=replay_buffer, buffer_ratio_retrain=1.0, length_episode=32, max_planning_steps=5)

print('\n\nTotal time to run the program is ' + str((time.time() - start_time) / 60) + ' minutes')



0th round of the main training process:
    interaction with 0th user in this round finished
    interaction with 99th user in this round finished
    interaction with 199th user in this round finished
    interaction with 299th user in this round finished
    interaction with 399th user in this round finished
    interaction with 499th user in this round finished
    interaction with 599th user in this round finished
    interaction with 699th user in this round finished
    interaction with 799th user in this round finished
    interaction with 899th user in this round finished
    interaction with 999th user in this round finished
    interaction with 1099th user in this round finished
    interaction with 1199th user in this round finished
    interaction with 1299th user in this round finished
    interaction with 1399th user in this round finished
    interaction with 1499th user in this round finished
    interaction with 1599th user in this round finished
    interaction with

    interaction with 99th user in this round finished
    interaction with 199th user in this round finished
    interaction with 299th user in this round finished
    interaction with 399th user in this round finished
    interaction with 499th user in this round finished
    interaction with 599th user in this round finished
    interaction with 699th user in this round finished
    interaction with 799th user in this round finished
    interaction with 899th user in this round finished
    interaction with 999th user in this round finished
    interaction with 1099th user in this round finished
    interaction with 1199th user in this round finished
    interaction with 1299th user in this round finished
    interaction with 1399th user in this round finished
    interaction with 1499th user in this round finished
    interaction with 1599th user in this round finished
    interaction with 1699th user in this round finished
    interaction with 1799th user in this round finished
   

    interaction with 199th user in this round finished
    interaction with 299th user in this round finished
    interaction with 399th user in this round finished
    interaction with 499th user in this round finished
    interaction with 599th user in this round finished
    interaction with 699th user in this round finished
    interaction with 799th user in this round finished
    interaction with 899th user in this round finished
    interaction with 999th user in this round finished
    interaction with 1099th user in this round finished
    interaction with 1199th user in this round finished
    interaction with 1299th user in this round finished
    interaction with 1399th user in this round finished
    interaction with 1499th user in this round finished
    interaction with 1599th user in this round finished
    interaction with 1699th user in this round finished
    interaction with 1799th user in this round finished
    interaction with 1899th user in this round finished
 

    interaction with 299th user in this round finished
    interaction with 399th user in this round finished
    interaction with 499th user in this round finished
    interaction with 599th user in this round finished
    interaction with 699th user in this round finished
    interaction with 799th user in this round finished
    interaction with 899th user in this round finished
    interaction with 999th user in this round finished
    interaction with 1099th user in this round finished
    interaction with 1199th user in this round finished
    interaction with 1299th user in this round finished
    interaction with 1399th user in this round finished
    interaction with 1499th user in this round finished
    interaction with 1599th user in this round finished
    interaction with 1699th user in this round finished
    interaction with 1799th user in this round finished
    interaction with 1899th user in this round finished
    interaction with 1999th user in this round finished


    interaction with 499th user in this round finished
    interaction with 599th user in this round finished
    interaction with 699th user in this round finished
    interaction with 799th user in this round finished
    interaction with 899th user in this round finished
    interaction with 999th user in this round finished
    interaction with 1099th user in this round finished
    interaction with 1199th user in this round finished
    interaction with 1299th user in this round finished
    interaction with 1399th user in this round finished
    interaction with 1499th user in this round finished
    interaction with 1599th user in this round finished
    interaction with 1699th user in this round finished
    interaction with 1799th user in this round finished
    interaction with 1899th user in this round finished
    interaction with 1999th user in this round finished
    interaction with 2099th user in this round finished
    interaction with 2199th user in this round finishe

...

    interaction with 399th user in this round finished
    interaction with 499th user in this round finished
    interaction with 599th user in this round finished
    interaction with 699th user in this round finished
    interaction with 799th user in this round finished
    interaction with 899th user in this round finished
    interaction with 999th user in this round finished
    interaction with 1099th user in this round finished
    interaction with 1199th user in this round finished
    interaction with 1299th user in this round finished
    interaction with 1399th user in this round finished
    interaction with 1499th user in this round finished
    interaction with 1599th user in this round finished
    interaction with 1699th user in this round finished
    interaction with 1799th user in this round finished
    interaction with 1899th user in this round finished
    interaction with 1999th user in this round finished
    interaction with 2099th user in this round finished

    interaction with 799th user in this round finished
    interaction with 899th user in this round finished
    interaction with 999th user in this round finished
    interaction with 1099th user in this round finished
    interaction with 1199th user in this round finished
    interaction with 1299th user in this round finished
    interaction with 1399th user in this round finished
    interaction with 1499th user in this round finished
    interaction with 1599th user in this round finished
    interaction with 1699th user in this round finished
    interaction with 1799th user in this round finished
    interaction with 1899th user in this round finished
    interaction with 1999th user in this round finished
    interaction with 2099th user in this round finished
    interaction with 2199th user in this round finished
    interaction with 2299th user in this round finished
    interaction with 2399th user in this round finished
    interaction with 2499th user in this round fini

    interaction with 799th user in this round finished
    interaction with 899th user in this round finished
    interaction with 999th user in this round finished
    interaction with 1099th user in this round finished
    interaction with 1199th user in this round finished
    interaction with 1299th user in this round finished
    interaction with 1399th user in this round finished
    interaction with 1499th user in this round finished
    interaction with 1599th user in this round finished
    interaction with 1699th user in this round finished
    interaction with 1799th user in this round finished
    interaction with 1899th user in this round finished
    interaction with 1999th user in this round finished
    interaction with 2099th user in this round finished
    interaction with 2199th user in this round finished
    interaction with 2299th user in this round finished
    interaction with 2399th user in this round finished
    interaction with 2499th user in this round fini

    interaction with 999th user in this round finished
    interaction with 1099th user in this round finished
    interaction with 1199th user in this round finished
    interaction with 1299th user in this round finished
    interaction with 1399th user in this round finished
    interaction with 1499th user in this round finished
    interaction with 1599th user in this round finished
    interaction with 1699th user in this round finished
    interaction with 1799th user in this round finished
    interaction with 1899th user in this round finished
    interaction with 1999th user in this round finished
    interaction with 2099th user in this round finished
    interaction with 2199th user in this round finished
    interaction with 2299th user in this round finished
    interaction with 2399th user in this round finished
    interaction with 2499th user in this round finished
    interaction with 2599th user in this round finished
    interaction with 2699th user in this round fi

    interaction with 1099th user in this round finished
    interaction with 1199th user in this round finished
    interaction with 1299th user in this round finished
    interaction with 1399th user in this round finished
    interaction with 1499th user in this round finished
    interaction with 1599th user in this round finished
    interaction with 1699th user in this round finished
    interaction with 1799th user in this round finished
    interaction with 1899th user in this round finished
    interaction with 1999th user in this round finished
    interaction with 2099th user in this round finished
    interaction with 2199th user in this round finished
    interaction with 2299th user in this round finished
    interaction with 2399th user in this round finished
    interaction with 2499th user in this round finished
    interaction with 2599th user in this round finished
    interaction with 2699th user in this round finished
    interaction with 2799th user in this round f

Remaining Issues:

~~`Question:`~~ When updating the policy network parameters, do we update the state representation model as well?

~~`Question:`~~ Similarly, when we update the state-value prediction network, do we update the state representation model?

**Answer:** it is unknown. Maybe try both! Currently we don't.
    
~~`Question:`~~ What about the discounting factor $\gamma$? Do we use it in our update?

**Answer:** Don't use it at this moment. Maybe Try it if I have time! 
    
`Question:` When t goes up, the user's historical records increases (ob becomes a long long list), which affects the speed of computing many functions needed to update the policy. Is it reasonable for us to keep track of only the most recent K records as our observation?

**Answer:** No, since the observation sequence won't get too long.    

~~`Question:`~~ When we fill in the replay buffer with interactive experience with the user, a lot of (simulated) rewards are 0 not because they're actually 0 but rather because they are unknown and we artificially set it to 0. Then when we use the replay buffer to train our user model (the reward prediction model), we are forcing the model to learn that the rewards for those actions are 0's, which might not be the case. We can try to use only experiences with real ratings to train the user model, but I am now not sure how to do it. Anyway, let's just do the former solution first and see what happens.

~~`Question:`~~ Parameter tuning

~~`Question:`~~ case study to understand its behavior

`Question:` User model overfitting (poor generalization?)

~~`Question:`~~ Try multi-step methods or eligibility trace for the policy-gradient method.

~~`Question:`~~ What about a model-free solution but with experience replay?

`Question:` $R_0$ might should be 2.5 rather than 0, because after normalization, $R_0$ becomes -1.

~~`Question:`~~ Are precision@n, recall@n, f1@n really reasonable evaluation metrics?

**Answer:** precision@n: what if some user has just a few (say < n) relevant items, then precision@n for this user will never be able to get close to 1 no matter how good the recommendation policy is.

recall@n: what if some user has NO relevant items (denominator is 0), should we set the recall@n for this user to 1, as we do in conventional recommendation systems? Also, the denominator is related to number of relevant items of the user, rather than related to n, so the range of recall@n is no longer [0,1] depending on the quality of the recommendation policy. Thus, I personally think it is no longer suitable metrics for our problem.

f1@n: if recall@n does not make sense in the first place, how can f1@n, one quantity that relies on precision@n and recall@n, make sense?